# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd
import numpy as np


cols= ['target', 'ids', 'date', 'flag', 'user', 'text']

df = pd.read_csv('training.csv', encoding='latin-1')

df.columns = cols

df.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [2]:
data = df.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here

import re

def clean_up(s): 
    element = re.sub('http://.+', '', s)
    element = re.sub('[^a-zA-Z0-9]', ' ', element)
    return (re.sub('\d+',' ',element)).lower()

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize(s):
    return word_tokenize(s)


from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist


def lemmatize(l):
  
    lem = WordNetLemmatizer()
    lemmatized = [lem.lemmatize(w,get_wordnet_pos(w)) for w in l]
    
    return lemmatized

from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(l):
    
    filtered_sentence = []
    
    for w in l:
        if w not in stopwords.words('english'):
            filtered_sentence.append(w)
    
    return filtered_sentence

[nltk_data] Downloading package punkt to /home/jorgebra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jorgebra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jorgebra/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jorgebra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data['text_processed'] = data['text'].apply(clean_up)
data.head()

,target,ids,date,flag,user,text,text_processed
464842,0,2175214973,Mon Jun 15 00:23:03 PDT 2009,NO_QUERY,izzyaezy,"is home.., gonna shave my goatee now.. NO MOR...",is home gonna shave my goatee now no mor...
127494,0,1834727124,Mon May 18 04:00:12 PDT 2009,NO_QUERY,merejames,@softlysoftly oh you poor thing! there is noth...,softlysoftly oh you poor thing there is noth...
1060484,4,1963482873,Fri May 29 12:47:27 PDT 2009,NO_QUERY,Trinity_B,@Votney_925 I see. Just finished fixin muh hair.,votney i see just finished fixin muh hair
303754,0,1999558770,Mon Jun 01 20:25:46 PDT 2009,NO_QUERY,LaceFacee,Bad day at least my phone is working for the...,bad day at least my phone is working for the...
1118676,4,1973639336,Sat May 30 11:32:38 PDT 2009,NO_QUERY,cathyelcira,i had rib sauce all over my hands and the phon...,i had rib sauce all over my hands and the phon...


In [5]:
data['text_processed'] = data['text_processed'].apply(tokenize)
data.head()

,target,ids,date,flag,user,text,text_processed
464842,0,2175214973,Mon Jun 15 00:23:03 PDT 2009,NO_QUERY,izzyaezy,"is home.., gonna shave my goatee now.. NO MOR...","[is, home, gon, na, shave, my, goatee, now, no..."
127494,0,1834727124,Mon May 18 04:00:12 PDT 2009,NO_QUERY,merejames,@softlysoftly oh you poor thing! there is noth...,"[softlysoftly, oh, you, poor, thing, there, is..."
1060484,4,1963482873,Fri May 29 12:47:27 PDT 2009,NO_QUERY,Trinity_B,@Votney_925 I see. Just finished fixin muh hair.,"[votney, i, see, just, finished, fixin, muh, h..."
303754,0,1999558770,Mon Jun 01 20:25:46 PDT 2009,NO_QUERY,LaceFacee,Bad day at least my phone is working for the...,"[bad, day, at, least, my, phone, is, working, ..."
1118676,4,1973639336,Sat May 30 11:32:38 PDT 2009,NO_QUERY,cathyelcira,i had rib sauce all over my hands and the phon...,"[i, had, rib, sauce, all, over, my, hands, and..."


In [6]:
data['text_processed'] = data['text_processed'].apply(lemmatize)
data.head()

,target,ids,date,flag,user,text,text_processed
464842,0,2175214973,Mon Jun 15 00:23:03 PDT 2009,NO_QUERY,izzyaezy,"is home.., gonna shave my goatee now.. NO MOR...","[be, home, gon, na, shave, my, goatee, now, no..."
127494,0,1834727124,Mon May 18 04:00:12 PDT 2009,NO_QUERY,merejames,@softlysoftly oh you poor thing! there is noth...,"[softlysoftly, oh, you, poor, thing, there, be..."
1060484,4,1963482873,Fri May 29 12:47:27 PDT 2009,NO_QUERY,Trinity_B,@Votney_925 I see. Just finished fixin muh hair.,"[votney, i, see, just, finish, fixin, muh, hair]"
303754,0,1999558770,Mon Jun 01 20:25:46 PDT 2009,NO_QUERY,LaceFacee,Bad day at least my phone is working for the...,"[bad, day, at, least, my, phone, be, work, for..."
1118676,4,1973639336,Sat May 30 11:32:38 PDT 2009,NO_QUERY,cathyelcira,i had rib sauce all over my hands and the phon...,"[i, have, rib, sauce, all, over, my, hand, and..."


In [7]:
data['text_processed'] = data['text_processed'].apply(remove_stopwords)
data.head()

,target,ids,date,flag,user,text,text_processed
464842,0,2175214973,Mon Jun 15 00:23:03 PDT 2009,NO_QUERY,izzyaezy,"is home.., gonna shave my goatee now.. NO MOR...","[home, gon, na, shave, goatee, goatee, anymore..."
127494,0,1834727124,Mon May 18 04:00:12 PDT 2009,NO_QUERY,merejames,@softlysoftly oh you poor thing! there is noth...,"[softlysoftly, oh, poor, thing, nothing, bad, ..."
1060484,4,1963482873,Fri May 29 12:47:27 PDT 2009,NO_QUERY,Trinity_B,@Votney_925 I see. Just finished fixin muh hair.,"[votney, see, finish, fixin, muh, hair]"
303754,0,1999558770,Mon Jun 01 20:25:46 PDT 2009,NO_QUERY,LaceFacee,Bad day at least my phone is working for the...,"[bad, day, least, phone, work, time]"
1118676,4,1973639336,Sat May 30 11:32:38 PDT 2009,NO_QUERY,cathyelcira,i had rib sauce all over my hands and the phon...,"[rib, sauce, hand, phone, rang, didnt, know, u..."


In [8]:
data['target'].value_counts()

0    10037
4     9963
Name: target, dtype: int64

In [9]:
data['target'] = data['target'].apply(lambda x: True if x == 4 else False)

data.head()

,target,ids,date,flag,user,text,text_processed
464842,False,2175214973,Mon Jun 15 00:23:03 PDT 2009,NO_QUERY,izzyaezy,"is home.., gonna shave my goatee now.. NO MOR...","[home, gon, na, shave, goatee, goatee, anymore..."
127494,False,1834727124,Mon May 18 04:00:12 PDT 2009,NO_QUERY,merejames,@softlysoftly oh you poor thing! there is noth...,"[softlysoftly, oh, poor, thing, nothing, bad, ..."
1060484,True,1963482873,Fri May 29 12:47:27 PDT 2009,NO_QUERY,Trinity_B,@Votney_925 I see. Just finished fixin muh hair.,"[votney, see, finish, fixin, muh, hair]"
303754,False,1999558770,Mon Jun 01 20:25:46 PDT 2009,NO_QUERY,LaceFacee,Bad day at least my phone is working for the...,"[bad, day, least, phone, work, time]"
1118676,True,1973639336,Sat May 30 11:32:38 PDT 2009,NO_QUERY,cathyelcira,i had rib sauce all over my hands and the phon...,"[rib, sauce, hand, phone, rang, didnt, know, u..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [10]:
# your code here

from nltk.probability import FreqDist
import itertools


blob = data['text_processed'].tolist()

blob = itertools.chain.from_iterable(blob)

fdist = FreqDist(blob)

common5000 = dict(fdist.most_common(5000))

most_common = list(common5000.keys())

most_common

['get',
 'go',
 'day',
 'good',
 'work',
 'like',
 'love',
 'quot',
 'u',
 'today',
 'time',
 'well',
 'lol',
 'back',
 'see',
 'one',
 'miss',
 'want',
 'know',
 'think',
 'make',
 'really',
 'im',
 'feel',
 'amp',
 'come',
 'need',
 'night',
 'new',
 'still',
 'home',
 'oh',
 'watch',
 'thanks',
 'na',
 'look',
 'twitter',
 'great',
 'much',
 'say',
 'hope',
 'morning',
 'bad',
 'sleep',
 'wish',
 'wait',
 'take',
 'last',
 'tomorrow',
 'sad',
 'try',
 'haha',
 'happy',
 'would',
 'fun',
 'week',
 'thing',
 'tonight',
 'right',
 'friend',
 'way',
 'sorry',
 'nice',
 'could',
 'though',
 'follow',
 'yeah',
 'start',
 'gon',
 'bed',
 'yes',
 'even',
 'people',
 'hate',
 'hour',
 'school',
 'guy',
 'tweet',
 'weekend',
 'long',
 'show',
 'hey',
 'cant',
 'little',
 'x',
 'next',
 'life',
 'never',
 'year',
 'tire',
 'movie',
 'let',
 'suck',
 'use',
 'everyone',
 'soon',
 'first',
 'already',
 'sick',
 'ok',
 'play',
 'thank',
 'lt',
 'ready',
 'best',
 'eat',
 'find',
 'dont',
 'call',

In [11]:
target = data['target']

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [12]:
# your code here


for_review = data['text_processed']

def find_features(document):
    words = set(document)
    features = {}
    for w in most_common:
        features[w] = (w in words)
    
    return features

X = for_review.map(find_features)

In [13]:
target_list = target.tolist()

target_list[0:10]

[False, False, True, False, True, True, False, True, False, True]

In [14]:
list_x = X.values.tolist()

list_x[0:10]

[{'get': False,
  'go': False,
  'day': False,
  'good': False,
  'work': False,
  'like': False,
  'love': False,
  'quot': False,
  'u': False,
  'today': False,
  'time': False,
  'well': False,
  'lol': False,
  'back': False,
  'see': False,
  'one': False,
  'miss': False,
  'want': False,
  'know': False,
  'think': False,
  'make': False,
  'really': False,
  'im': False,
  'feel': False,
  'amp': False,
  'come': False,
  'need': False,
  'night': False,
  'new': False,
  'still': False,
  'home': True,
  'oh': False,
  'watch': False,
  'thanks': False,
  'na': True,
  'look': False,
  'twitter': False,
  'great': False,
  'much': False,
  'say': False,
  'hope': False,
  'morning': False,
  'bad': False,
  'sleep': False,
  'wish': False,
  'wait': False,
  'take': False,
  'last': False,
  'tomorrow': False,
  'sad': False,
  'try': False,
  'haha': False,
  'happy': False,
  'would': False,
  'fun': False,
  'week': False,
  'thing': False,
  'tonight': False,
  'right': F

In [15]:
X_total = []

for element in range(0,5001):
    X_total.append((list_x[element],target_list[element]))

In [16]:
len(X_total[0])

2

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [17]:
# your code here

import nltk

training_set = X_total[:3000]

testing_set = X_total[3000:]

In [18]:
len(X_total)

5001

In [19]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

classifier.show_most_informative_features(15)

Most Informative Features
                     sad = True            False : True   =     19.5 : 1.0
                    damn = True            False : True   =     11.7 : 1.0
                    poor = True            False : True   =     11.0 : 1.0
                    suck = True            False : True   =     10.2 : 1.0
                  thanks = True             True : False  =     10.2 : 1.0
                   sorry = True            False : True   =      8.1 : 1.0
                    lose = True            False : True   =      7.8 : 1.0
                computer = True            False : True   =      7.7 : 1.0
                   thank = True             True : False  =      7.4 : 1.0
                    sick = True            False : True   =      7.0 : 1.0
                    bore = True            False : True   =      7.0 : 1.0
                 welcome = True             True : False  =      7.0 : 1.0
            followfriday = True             True : False  =      6.3 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [20]:
# your code here

nltk.classify.accuracy(classifier, testing_set)

0.6931534232883558

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [21]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [22]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [23]:
# your code here